# Metallic Flat Plate Magnetized in Exterior Field 
# Test Case in TU Delft Master Thesis of Lisette de Bruin 
# 2D Toy Version 

<b>Goal</b>: The goal of this notebook is to compute a 2D numerical reference solution for the magnetization vector of metallic plate $\vec{M}(\vec{x})$ placed in homogeneous external magnetic field; 

## Import Packages

In [2]:
using Ferrite, SparseArrays

## Section 1: Using Ferrite.jl 

### Ferrite.jl Howto: What are cellvalues ? 

In [3]:
?generate_grid

search: generate_grid



```
generate_grid(celltype::Cell, nel::NTuple, [left::Vec, right::Vec)
```

Return a `Grid` for a rectangle in 1, 2 or 3 dimensions. `celltype` defined the type of cells, e.g. `Triangle` or `Hexahedron`. `nel` is a tuple of the number of elements in each direction. `left` and `right` are optional endpoints of the domain. Defaults to -1 and 1 in all directions.


In [6]:
# generate tiny grid for testing purposes 
nels  = (2, 2) # number of elements in each spatial direction
left  = Vec((0.0, 0.0)) # start point for geometry 
right = Vec((1.0, 1.0,))    # end point for geometry
grid = generate_grid(Quadrilateral, nels,left, right) 

Grid{2, Quadrilateral, Float64} with 4 Quadrilateral cells and 9 nodes

In [7]:
dim = 2
ip = Lagrange{dim, RefCube, 1}()
qr = QuadratureRule{dim, RefCube}(2)
cellvalues = CellScalarValues(qr, ip)

CellScalarValues{2, Float64, RefCube} with 4 shape functions and 4 quadrature points

### Ferrite.jl Howto: What does re-init do? 

In [8]:
?reinit!

search: reinit! atreplinit parentindices CartesianIndex CartesianIndices



```
reinit!(cv::CellValues, x::Vector)
reinit!(bv::FaceValues, x::Vector, face::Int)
```

Update the `CellValues`/`FaceValues` object for a cell or face with coordinates `x`. The derivatives of the shape functions, and the new integration weights are computed.


### Ferrite.jl Howto:  How to get coords and coord_qp?
Obtain coordinates of quadrature points in each cell by 
1. loop over cells in the mesh using CellIterator(dh);
2. for each cell, retrieve the coordinates for each cell using getcoordinates() yielding coord as output; 
3. reinit cellvalues for each cell using reinit!;   
4. loop over quad points of cell and retrieve spatial coordinate for each quad point in the cell using spatial_coordinate() and coords as input; 

In [9]:
dh = DofHandler(grid)
add!(dh, :u, 1)
close!(dh);

In [10]:
function my_diff_coeff(coord_qp)
    return coord_qp[1] + coord_qp[2]
end 

my_diff_coeff (generic function with 1 method)

In [12]:
function my_get_coords(dh::DofHandler)
    # Loop over all cels
    for (cellcount, cell) in enumerate(CellIterator(dh))
        coords = getcoordinates(cell)
        # display(coords)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            coords_qp = spatial_coordinate(cellvalues, q_point, coords)
            # display(my_diff_coeff(coords_qp)) 
        end
    end
    return 0
end

my_get_coords(dh) 

0

In [13]:
function my_get_coords(dh::DofHandler)
    # Loop over all cels
    for (cellcount, cell) in enumerate(CellIterator(dh))
        coords = getcoordinates(cell)
        # display(coords)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            coords_qp = spatial_coordinate(cellvalues, q_point, coords)
            # display(my_diff_coeff(coords_qp)) 
        end
    end
    return 0
end

my_get_coords (generic function with 1 method)

In [14]:
my_coords = my_get_coords(dh)
my_coords;

## Section 2: 2D Mockup of the Magnetized Plate  

In [15]:
# define spatially varying diffusion coefficient 
function my_diff_coeff(coord_qp)
    xbound = abs(coord_qp[1])<0.3
    ybound = abs(coord_qp[2])<0.1    
    return 1+1e6*(xbound*ybound) 
end 

my_diff_coeff (generic function with 1 method)

In [16]:
# Ke: added spatially varying diffusion coefficient 
# fe: forces zero source term 
function assemble_element!(Ke::Matrix, fe::Vector, cellvalues::CellScalarValues, mycoords)
    n_basefuncs = getnbasefunctions(cellvalues)
    # Reset to 0
    fill!(Ke, 0)
    fill!(fe, 0)
    # Loop over quadrature points
    for q_point in 1:getnquadpoints(cellvalues)
        # Get the quadrature weight
        dΩ = getdetJdV(cellvalues, q_point)
        # ADDED: Get coord of quadrature point
        coords_qp = spatial_coordinate(cellvalues, q_point, mycoords)
        # ADDED: Evaluate spatially dependent diffusion coefficient in quad point 
        val_diff_coeff = my_diff_coeff(coords_qp)
        # Loop over test shape functions
        for i in 1:n_basefuncs
            δu  = shape_value(cellvalues, q_point, i)
            ∇δu = shape_gradient(cellvalues, q_point, i)
            # Add contribution to fe
            fe[i] += 0 * δu * dΩ
            # Loop over trial shape functions
            for j in 1:n_basefuncs
                ∇u = shape_gradient(cellvalues, q_point, j)
                # MODIFIED: Add contribution to Ke
                Ke[i, j] += val_diff_coeff * (∇δu ⋅ ∇u) * dΩ
            end
        end
    end
    return Ke, fe
end

assemble_element! (generic function with 1 method)

In [17]:
function assemble_global(cellvalues::CellScalarValues, K::SparseMatrixCSC, dh::DofHandler)
    # Allocate the element stiffness matrix and element force vector
    n_basefuncs = getnbasefunctions(cellvalues)
    Ke = zeros(n_basefuncs, n_basefuncs)
    fe = zeros(n_basefuncs)
    # Allocate global force vector f
    f = zeros(ndofs(dh))
    # Create an assembler
    assembler = start_assemble(K, f)
    # Loop over all cels
    for cell in CellIterator(dh)
        # Added: Get coordinates from current cell 
        coords = getcoordinates(cell)
        # Reinitialize cellvalues for this cell
        reinit!(cellvalues, cell)
        # Modified - Compute element contribution
        assemble_element!(Ke, fe, cellvalues, coords)
        # Assemble Ke and fe into K and f
        assemble!(assembler, celldofs(cell), Ke, fe)
    end
    return K, f
end

assemble_global (generic function with 1 method)

In [141]:
nels  = (5, 5) # number of elements in each spatial direction
left  = Vec((-1.0, -1.0)) # start point for geometry 
right = Vec((1.0, 1.0,))    # end point for geometry
grid = generate_grid(Quadrilateral,nels,left,right);

# nels  = (100, 50) # number of elements in each spatial direction
# left  = Vec((0.0, 0.0)) # start point for geometry 
# right = Vec((2.0, 1.0,))    # end point for geometry
# grid = generate_grid(Quadrilateral,nels,left,right);
#grid 

In [142]:
dim = 2
ip = Lagrange{dim, RefCube, 1}()
qr = QuadratureRule{dim, RefCube}(2)
cellvalues = CellScalarValues(qr, ip);

In [143]:
dh = DofHandler(grid)
add!(dh, :u, 1)
close!(dh);

In [144]:
K = create_sparsity_pattern(dh);

In [145]:
ch = ConstraintHandler(dh);

In [146]:
∂Ω = union(
    getfaceset(grid, "left"),
    getfaceset(grid, "right"),
    getfaceset(grid, "top"),
    getfaceset(grid, "bottom"),
);

# ∂Ω = union(
#    getfaceset(grid, "right"),
#    getfaceset(grid, "top"),
# );

In [147]:
dbc = Dirichlet(:u, ∂Ω, (x, t) -> x[1])
add!(ch, dbc);

In [148]:
close!(ch)

ConstraintHandler:
  BCs:
    Field: u, Components: [1]

In [149]:
K, f = assemble_global(cellvalues, K, dh);

In [150]:
apply!(K, f, ch)
u = K \ f;

In [151]:
u

36-element Vector{Float64}:
 -1.0
 -0.6000000000000001
 -7.069147921125653e-6
 -1.0
 -0.19999999999999993
 -9.216068649105236e-7
  0.19999999999999993
  9.216068643896823e-7
  0.6000000000000001
  7.0691479206048145e-6
  1.0
  1.0
 -2.232044708101914e-6
  ⋮
 -7.069147921125656e-6
 -1.0
 -9.216068649105247e-7
  9.216068643896813e-7
  7.069147920604817e-6
  1.0
 -0.6000000000000001
 -1.0
 -0.19999999999999993
  0.19999999999999993
  0.6000000000000001
  1.0

In [152]:
vtk_grid("magnetic_plate", dh) do vtk
    vtk_point_data(vtk, dh, u)
end

1-element Vector{String}:
 "magnetic_plate.vtu"

### Computing Fluxes

In [153]:
function my_get_coords(dh::DofHandler)
    # Loop over all cels
    for (cellcount, cell) in enumerate(CellIterator(dh))
        coords = getcoordinates(cell)
        display(coords)
        reinit!(cellvalues, cell)
        for q_point in 1:getnquadpoints(cellvalues)
            coords_qp = spatial_coordinate(cellvalues, q_point, coords)
            # display(my_diff_coeff(coords_qp)) 
        end
    end
    return 0
end

my_get_coords (generic function with 1 method)

In [157]:
# define spatially varying diffusion coefficient 
function my_diff_coeff(coord_qp)
    xbound = abs(coord_qp[1])<0.3
    ybound = abs(coord_qp[2])<0.2 
    return 1+1e6*(xbound*ybound) 
end 

my_diff_coeff (generic function with 1 method)

In [158]:
function compute_hfield(cellvalues::CellScalarValues{dim,T}, dh::DofHandler, a) where {dim,T}

    n = getnbasefunctions(cellvalues)
    cell_dofs = zeros(Int, n)
    nqp = getnquadpoints(cellvalues)

    # Allocate storage for the fluxes to store
    q = [Vec{2,T}[] for _ in 1:getncells(dh.grid)]

    for (cell_num, cell) in enumerate(CellIterator(dh))
        q_cell = q[cell_num]
        celldofs!(cell_dofs, dh, cell_num)
        aᵉ = a[cell_dofs]
        reinit!(cellvalues, cell)

        for q_point in 1:nqp
            q_qp = - function_gradient(cellvalues, q_point, aᵉ)
            push!(q_cell, q_qp)
        end
    end
    return q
end

function compute_bfield(cellvalues::CellScalarValues{dim,T}, dh::DofHandler, a) where {dim,T}

    n = getnbasefunctions(cellvalues)
    cell_dofs = zeros(Int, n)
    nqp = getnquadpoints(cellvalues)

    # Allocate storage for the fluxes to store
    q = [Vec{2,T}[] for _ in 1:getncells(dh.grid)]

    for (cell_num, cell) in enumerate(CellIterator(dh))
        q_cell = q[cell_num]
        celldofs!(cell_dofs, dh, cell_num)
        aᵉ = a[cell_dofs]
        reinit!(cellvalues, cell)
        coords = getcoordinates(cell)
        
        for q_point in 1:nqp
            coords_qp = spatial_coordinate(cellvalues, q_point, coords)
            val_diff_coeff = my_diff_coeff(coords_qp)
            display(coords_qp)
            display(val_diff_coeff)
            q_qp = - val_diff_coeff*function_gradient(cellvalues, q_point, aᵉ)
            push!(q_cell, q_qp)
        end
    end
    return q
end

compute_bfield (generic function with 1 method)

In [159]:
h_gp = compute_hfield(cellvalues, dh, u);
b_gp = compute_bfield(cellvalues, dh, u);
#q_gp

2-element Vec{2, Float64}:
 -0.9154700538379252
 -0.9154700538379251

1.0

2-element Vec{2, Float64}:
 -0.684529946162075
 -0.9154700538379251

1.0

2-element Vec{2, Float64}:
 -0.9154700538379252
 -0.684529946162075

...

1.0

2-element Vec{2, Float64}:
 0.684529946162075
 0.9154700538379252

1.0

2-element Vec{2, Float64}:
 0.9154700538379251
 0.9154700538379252

1.0

In [162]:
[h_gp[i][1] for i=1:25]

25-element Vector{Vec{2, Float64}}:
 [-1.3169835633909486, -0.3169835633909486]
 [-0.7886783824155286, -1.2886607095457259]
 [-0.7886761083870458, -0.28867380436988366]
 [-0.7886783824155287, 0.7113193135673105]
 [-1.3169835633909481, 1.1829987637392503]
 [-2.4999848826306614, -2.5555004636856893e-6]
 [-1.2892116028171218e-5, -9.616021420192745e-6]
 [-4.450506620767515e-6, -2.151868433915737e-7]
 [-1.2892116028171227e-5, 2.8494511582411787e-6]
 [-2.499984882630662, 9.537257569250742e-6]
 [-2.499994419888231, -0.0]
 [-3.648809153853049e-6, -8.470329472543003e-22]
 [-3.8626052315013614e-6, 6.352747104407253e-22]
 [-3.648809153853048e-6, 1.6940658945086007e-21]
 [-2.4999944198882313, 3.361966941276516e-16]
 [-2.4999918643877663, 2.5555004636856893e-6]
 [-6.125545766219649e-6, 9.61602142019275e-6]
 [-4.020132933984363e-6, 2.1518684339157602e-7]
 [-6.125545766219648e-6, -2.8494511582411796e-6]
 [-2.4999918643877668, -9.537257568615064e-6]
 [-2.1829987637392496, 0.31698356339094835]
 [-0.211

In [164]:
[h_gp[i][1] - b_gp[i][1] for i=1:25]

25-element Vector{Vec{2, Float64}}:
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [3.8626052315013615, -6.352747104407253e-16]
 [3.6488091538530476, -1.6940658945086007e-15]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]
 [0.0, 0.0]

In [165]:
projector = L2Projector(ip, grid);
#projector 

In [166]:
#?project

In [167]:
h_projected = project(projector, h_gp, qr; project_to_nodes=false);
b_projected = project(projector, b_gp, qr; project_to_nodes=false);
# q_projected

### Exporting to VTK 

In [168]:
?vtk_point_data

search: vtk_point_data



```
vtk_point_data(vtk, data::Vector{<:AbstractTensor}, name)
```

Write the tensor field `data` to the vtk file. Two-dimensional tensors are padded with zeros.

For second order tensors the following indexing ordering is used: `[11, 22, 33, 23, 13, 12, 32, 31, 21]`. This is the default Voigt order in Tensors.jl.


In [169]:
vtk_grid("magnetic_plate", dh) do vtk
    vtk_point_data(vtk, dh, u)
    vtk_point_data(vtk, projector, h_projected, "H")
    vtk_point_data(vtk, projector, b_projected, "B")
end

1-element Vector{String}:
 "magnetic_plate.vtu"